In [1]:
#%%capture
#!pip install py-heat-magic

#%load_ext heat

In [2]:
import numpy as np
import networkx as nx
from tqdm.notebook import tqdm
import pickle
import time
import torch
from torch import nn
cuda = torch.device('cuda')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

project_path = "/content/drive/MyDrive/Colab Notebooks/Projects/PIE/"

Mounted at /content/drive


In [4]:
def load_graph(lang1, lang2):
    with open(project_path + "Graphs/" + str(lang1) + "_" + str(lang2) + "_" + "graph.pickle", 'rb') as f:
        return pickle.load(f)

In [5]:
def make_tensors(graph):
    return {k : torch.FloatTensor(v).to(cuda) for k, v in graph.items()}

In [29]:
hi_de_graph = make_tensors(load_graph('hi', 'de'))
hi_en_graph = make_tensors(load_graph('hi', 'en'))
hi_fr_graph = make_tensors(load_graph('hi', 'fr'))
hi_la_graph = make_tensors(load_graph('hi', 'la'))
hi_sp_graph = make_tensors(load_graph('hi', 'sp'))

In [ ]:
ma_de_graph = make_tensors(load_graph('ma', 'de'))
ma_en_graph = make_tensors(load_graph('ma', 'en'))
ma_fr_graph = make_tensors(load_graph('ma', 'fr'))
ma_la_graph = make_tensors(load_graph('ma', 'la'))
ma_sp_graph = make_tensors(load_graph('ma', 'sp'))

In [ ]:
pa_de_graph = make_tensors(load_graph('pa', 'de'))
pa_en_graph = make_tensors(load_graph('pa', 'en'))
pa_fr_graph = make_tensors(load_graph('pa', 'fr'))
pa_la_graph = make_tensors(load_graph('pa', 'la'))
pa_sp_graph = make_tensors(load_graph('pa', 'sp'))

In [ ]:
ta_de_graph = make_tensors(load_graph('ta', 'de'))
ta_en_graph = make_tensors(load_graph('ta', 'en'))
ta_fr_graph = make_tensors(load_graph('ta', 'fr'))
ta_la_graph = make_tensors(load_graph('ta', 'la'))
ta_sp_graph = make_tensors(load_graph('ta', 'sp'))

In [9]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)

In [10]:
def sims_to_other_nodes(node, graph):
    sims = []
    for k, v in graph.items():
        sim = round(cos(graph[node], v).item(), 5)
        sims.append(sim)
        #sims.update({k : sim})
    return sims #, sum(list(sims.values()))

In [11]:
def get_all_sims(graph):
    all_sims = []#{}
    for node, v in tqdm(graph.items()):
        sims = sims_to_other_nodes(node, graph)
        all_sims.append(sims) #update(sims)
    return all_sims

In [ ]:
%%capture
!pip install pipe

from pipe import traverse

In [24]:
def create_adjacency_list(graph):
    adj_l = {}
    sims = list(get_all_sims(graph) | traverse)
    cossim90pc = np.percentile(sims, 90)
    for node, vector in tqdm(graph.items()):
        for k, v in graph.items():
            if round(cos(graph[node], v).item(), 5) >= cossim90pc:
                adj_l.update({node : k})
    return adj_l

In [33]:
def save_adj(adj, lang1, lang2):
    f =  open(project_path + "Results/" + lang1 + "_" + lang2 + "_adj.txt", "w")
    for k, v in adj.items():
        f.writelines(str(k) + ' ' + str(v) +'\n')
    f.close()

In [ ]:
save_adj(create_adjacency_list(hi_de_graph), 'hi', 'de')
save_adj(create_adjacency_list(hi_en_graph), 'hi', 'en')
save_adj(create_adjacency_list(hi_fr_graph), 'hi', 'fr')
save_adj(create_adjacency_list(hi_la_graph), 'hi', 'la')
save_adj(create_adjacency_list(hi_sp_graph), 'hi', 'sp')

In [ ]:
save_adj(create_adjacency_list(ma_de_graph), 'ma', 'de')
save_adj(create_adjacency_list(ma_en_graph), 'ma', 'en')
save_adj(create_adjacency_list(ma_fr_graph), 'ma', 'fr')
save_adj(create_adjacency_list(ma_la_graph), 'ma', 'la')
save_adj(create_adjacency_list(ma_sp_graph), 'ma', 'sp')

In [ ]:
save_adj(create_adjacency_list(pa_de_graph), 'pa', 'de')
save_adj(create_adjacency_list(pa_en_graph), 'pa', 'en')
save_adj(create_adjacency_list(pa_fr_graph), 'pa', 'fr')
save_adj(create_adjacency_list(pa_la_graph), 'pa', 'la')
save_adj(create_adjacency_list(pa_sp_graph), 'pa', 'sp')

In [ ]:
save_adj(create_adjacency_list(ta_de_graph), 'ta', 'de')
save_adj(create_adjacency_list(ta_en_graph), 'ta', 'en')
save_adj(create_adjacency_list(ta_fr_graph), 'ta', 'fr')
save_adj(create_adjacency_list(ta_la_graph), 'ta', 'la')
save_adj(create_adjacency_list(ta_sp_graph), 'ta', 'sp')